In [62]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from os.path import join as opj
import keras

#from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
#import pylab
#plt.rcParams['figure.figsize'] = 10, 10
#%matplotlib inline

In [63]:
train = pd.read_json("../_RawData/train.json/data/processed/train.json")
target_train=train['is_iceberg']
test = pd.read_json("../_RawData/test.json/data/processed/test.json")


In [64]:
# old_results = pd.read_csv("subvgg16mobile.csv")
pseudo_data = pd.read_csv("pseudo_data.csv")

In [65]:
pseudo_data.head()

,id,is_iceberg
0,b20200e4,0.000100
1,e7f018bb,0.999509
2,92a51ffb,0.999900
3,565b28ac,0.000100
4,1fae4879,0.000100


In [70]:
train.head()

,band_1,band_2,id,inc_angle,is_iceberg
0,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",dfd5f913,43.9239,0
1,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",e25388fd,38.1562,0
2,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",58b2aaa0,45.2859,1
3,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",4cfc3a18,43.8306,0
4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",271f93f4,35.6256,0


In [68]:
# test[test["id"] == pseudo_data["id"]]
pseudo_train = pd.DataFrame(test[test["id"].isin(pseudo_data["id"])])
pseudo_train = pseudo_train.reset_index(drop = True)
pseudo_train["is_iceberg"] = np.where(pseudo_data["is_iceberg"] < 0.5, 0, 1)


In [69]:
pseudo_train.head()

,band_1,band_2,id,inc_angle,is_iceberg
0,"[-14.14109992980957, -15.064241409301758, -17....","[-14.74563980102539, -14.590410232543945, -14....",b20200e4,37.505433,0
1,"[-12.167478, -13.706167, -16.54837, -13.572674...","[-24.32222, -26.375538, -24.096739, -23.8769, ...",e7f018bb,34.473900,1
2,"[-20.162964, -18.16898, -17.023144, -17.023216...","[-30.905346, -28.406647, -25.381365, -28.77645...",92a51ffb,33.638300,1
3,"[-17.691271, -16.036381, -17.908293, -22.23594...","[-26.91781, -24.979542, -24.979473, -24.734716...",565b28ac,35.782900,0
4,"[-18.200674, -18.327515, -18.990921, -19.56073...","[-22.108603, -23.156403, -23.382061, -24.60749...",1fae4879,45.247800,0


In [71]:
print(train.shape)
print(pseudo_train.shape)

(1604, 5)
(1198, 5)


In [72]:
print(pseudo_train[pseudo_train.is_iceberg == True].shape)
print(pseudo_train[pseudo_train.is_iceberg == False].shape)


(334, 5)
(864, 5)


In [73]:
is_iceberg_true = pseudo_train[pseudo_train.is_iceberg == True]

In [74]:
is_iceberg_false = pseudo_train[pseudo_train.is_iceberg == False][:334]

In [75]:
is_iceberg_all = is_iceberg_true.append(is_iceberg_false)

In [76]:
is_iceberg_all = is_iceberg_all.sample(frac=1).reset_index(drop=True)

In [77]:
is_iceberg_all.shape

(668, 5)

In [78]:
train = train.append(is_iceberg_all)
train = train.reset_index(drop=True)

In [79]:
train.shape

(2272, 5)

In [80]:
target_train=train['is_iceberg']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle']=pd.to_numeric(train['inc_angle'], errors='coerce')#We have only 133 NAs.
train['inc_angle']=train['inc_angle'].fillna(method='pad')
X_angle=train['inc_angle']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
X_test_angle=test['inc_angle']

#Generate the training data
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
#X_band_3=(X_band_1+X_band_2)/2
X_band_3=np.fabs(np.subtract(X_band_1,X_band_2))
X_band_4=np.maximum(X_band_1,X_band_2)
X_band_5=np.minimum(X_band_1,X_band_2)
#X_band_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in train["inc_angle"]])
X_train = np.concatenate([
                          
                          X_band_3[:, :, :, np.newaxis],X_band_4[:, :, :, np.newaxis],X_band_5[:, :, :, np.newaxis]], axis=-1)



X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
#X_band_test_3=(X_band_test_1+X_band_test_2)/2
X_band_test_3=np.fabs(np.subtract(X_band_test_1,X_band_test_2))
X_band_test_4=np.maximum(X_band_test_1,X_band_test_2)
X_band_test_5=np.minimum(X_band_test_1,X_band_test_2)
#X_band_test_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in test["inc_angle"]])
X_test = np.concatenate([
                          X_band_test_3[:, :, :, np.newaxis], X_band_test_4[:, :, :, np.newaxis],X_band_test_5[:, :, :, np.newaxis]],axis=-1)


In [81]:
#Import Keras.
#from matplotlib import pyplot
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import *
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.optimizers import rmsprop
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

from keras.datasets import cifar10
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input	

#Data Aug for multi-input
from keras.preprocessing.image import ImageDataGenerator
batch_size=64
# Define the image transformations here
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.,
                         height_shift_range = 0.,
                         channel_shift_range=0,
                         zoom_range = 0.5,
                         rotation_range = 10)

# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i[1]], X1i[1]

# Finally create generator
def get_callbacks(filepath, patience=2):
   #es = EarlyStopping('val_loss', patience=10, mode="min")
   es = EarlyStopping('val_loss', patience=20, mode="min")
   msave = ModelCheckpoint(filepath, save_best_only=True)
   return [es, msave]


def getVggAngleModel():
    input_2 = Input(shape=[1], name="angle")
    angle_layer = Dense(1, )(input_2)
    base_model = VGG16(weights='imagenet', include_top=False, 
                 input_shape=X_train.shape[1:], classes=1)
    x = base_model.get_layer('block5_pool').output
    x = GlobalMaxPooling2D()(x)
    base_model2 = keras.applications.mobilenet.MobileNet(weights=None, alpha=0.9,input_tensor = base_model.input,include_top=False, input_shape=X_train.shape[1:])

    x2 = base_model2.output
    x2 = GlobalAveragePooling2D()(x2)

    merge_one = concatenate([x, x2, angle_layer])

    merge_one = Dropout(0.6)(merge_one)
    predictions = Dense(1, activation='sigmoid',kernel_initializer='he_normal')(merge_one)
    
    model = Model(input=[base_model.input, input_2], output=predictions)
    
    sgd = Adam(lr=1e-4) #SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    return model


#Using K-fold Cross Validation with Data Augmentation.
def myAngleCV(X_train, X_angle, X_test):
    K=5
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(X_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n===================FOLD=',j)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx]
        Y_holdout= target_train[test_idx]
        
        #Angle
        X_angle_cv=X_angle[train_idx]
        X_angle_hold=X_angle[test_idx]

        #define file path and get callbacks
        file_path = "%s_aug_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=10)
        gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
        galaxyModel= getVggAngleModel()
        galaxyModel.fit_generator(
                gen_flow,
                steps_per_epoch=24,
                epochs=100,
                shuffle=True,
                verbose=1,
                validation_data=([X_holdout,X_angle_hold], Y_holdout),
                callbacks=callbacks)

        #Getting the Best Model
        galaxyModel.load_weights(filepath=file_path)
        #Getting Training Score
        score = galaxyModel.evaluate([X_train_cv,X_angle_cv], y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = galaxyModel.evaluate([X_holdout,X_angle_hold], Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        #Getting validation Score.
        pred_valid=galaxyModel.predict([X_holdout,X_angle_hold])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=galaxyModel.predict([X_test, X_test_angle])
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=galaxyModel.predict([X_train, X_angle])
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_test_pred_log=y_test_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

In [82]:
preds=myAngleCV(X_train, X_angle, X_test)
#Submission for each day.
submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=preds
submission.to_csv('subvgg16mobile_pseudo.csv', index=False)


===================FOLD= 0


C:\Users\Dave\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=[<tf.Tenso...)`


Epoch 1/100
24/24 [==============================] - 24s 1s/step - loss: 0.7798 - acc: 0.5964 - val_loss: 0.6895 - val_acc: 0.4989
Epoch 2/100
24/24 [==============================] - 10s 396ms/step - loss: 0.6856 - acc: 0.6370 - val_loss: 0.6873 - val_acc: 0.5209
Epoch 3/100
24/24 [==============================] - 10s 402ms/step - loss: 0.6440 - acc: 0.6318 - val_loss: 0.6722 - val_acc: 0.5956
Epoch 4/100
24/24 [==============================] - 10s 417ms/step - loss: 0.6005 - acc: 0.6621 - val_loss: 0.4805 - val_acc: 0.7780
Epoch 5/100
24/24 [==============================] - 8s 321ms/step - loss: 0.6154 - acc: 0.6569 - val_loss: 0.7111 - val_acc: 0.4791
Epoch 6/100
24/24 [==============================] - 8s 329ms/step - loss: 0.6212 - acc: 0.6543 - val_loss: 0.7276 - val_acc: 0.4791
Epoch 7/100
24/24 [==============================] - 8s 323ms/step - loss: 0.6088 - acc: 0.6699 - val_loss: 0.7372 - val_acc: 0.4791
Epoch 8/100
24/24 [==============================] - 8s 321ms/step -

24/24 [==============================] - 8s 327ms/step - loss: 0.1746 - acc: 0.9260 - val_loss: 0.1560 - val_acc: 0.9363
Epoch 38/100
24/24 [==============================] - 8s 327ms/step - loss: 0.1406 - acc: 0.9332 - val_loss: 0.1415 - val_acc: 0.9429
Epoch 39/100
24/24 [==============================] - 8s 327ms/step - loss: 0.1644 - acc: 0.9365 - val_loss: 0.1569 - val_acc: 0.9407
Epoch 40/100
24/24 [==============================] - 8s 328ms/step - loss: 0.1608 - acc: 0.9358 - val_loss: 0.2392 - val_acc: 0.9165
Epoch 41/100
24/24 [==============================] - 8s 332ms/step - loss: 0.2012 - acc: 0.9160 - val_loss: 0.1649 - val_acc: 0.9275
Epoch 42/100
24/24 [==============================] - 8s 328ms/step - loss: 0.1655 - acc: 0.9348 - val_loss: 0.1783 - val_acc: 0.9231
Train loss: 0.138264120886
Train accuracy: 0.93891029169
Test loss: 0.138463660931
Test accuracy: 0.940659340659

===================FOLD= 2
Epoch 1/100
24/24 [==============================] - 26s 1s/step - l

24/24 [==============================] - 9s 358ms/step - loss: 0.1548 - acc: 0.9357 - val_loss: 0.1360 - val_acc: 0.9449
Epoch 56/100
24/24 [==============================] - 8s 347ms/step - loss: 0.1441 - acc: 0.9350 - val_loss: 0.1402 - val_acc: 0.9339
Epoch 57/100
24/24 [==============================] - 8s 347ms/step - loss: 0.1394 - acc: 0.9463 - val_loss: 0.1313 - val_acc: 0.9471
Epoch 58/100
24/24 [==============================] - 8s 353ms/step - loss: 0.1327 - acc: 0.9424 - val_loss: 0.1516 - val_acc: 0.9405
Epoch 59/100
24/24 [==============================] - 8s 354ms/step - loss: 0.1383 - acc: 0.9453 - val_loss: 0.1385 - val_acc: 0.9427
Epoch 60/100
24/24 [==============================] - 8s 348ms/step - loss: 0.1400 - acc: 0.9421 - val_loss: 0.1650 - val_acc: 0.9317
Epoch 61/100
24/24 [==============================] - 9s 377ms/step - loss: 0.1411 - acc: 0.9359 - val_loss: 0.1292 - val_acc: 0.9515
Epoch 62/100
24/24 [==============================] - 8s 348ms/step - loss:

24/24 [==============================] - 9s 355ms/step - loss: 0.1393 - acc: 0.9303 - val_loss: 0.2572 - val_acc: 0.9097
Epoch 54/100
24/24 [==============================] - 8s 342ms/step - loss: 0.1379 - acc: 0.9399 - val_loss: 0.2585 - val_acc: 0.9185
Epoch 55/100
24/24 [==============================] - 8s 342ms/step - loss: 0.1572 - acc: 0.9391 - val_loss: 0.2584 - val_acc: 0.9141
Epoch 56/100
24/24 [==============================] - 8s 343ms/step - loss: 0.1569 - acc: 0.9402 - val_loss: 0.2461 - val_acc: 0.9031
Epoch 57/100
24/24 [==============================] - 9s 373ms/step - loss: 0.1471 - acc: 0.9264 - val_loss: 0.2276 - val_acc: 0.9163
Epoch 58/100
24/24 [==============================] - 8s 342ms/step - loss: 0.1430 - acc: 0.9348 - val_loss: 0.2034 - val_acc: 0.9229
Epoch 59/100
24/24 [==============================] - 9s 384ms/step - loss: 0.1282 - acc: 0.9434 - val_loss: 0.2147 - val_acc: 0.9273
Train loss: 0.110498280233
Train accuracy: 0.952695269527
Test loss: 0.1704

24/24 [==============================] - 8s 340ms/step - loss: 0.1295 - acc: 0.9476 - val_loss: 0.1828 - val_acc: 0.9295
Epoch 55/100
24/24 [==============================] - 8s 341ms/step - loss: 0.1521 - acc: 0.9407 - val_loss: 0.1622 - val_acc: 0.9273
Epoch 56/100
24/24 [==============================] - 9s 367ms/step - loss: 0.1324 - acc: 0.9467 - val_loss: 0.1934 - val_acc: 0.9317
Epoch 57/100
24/24 [==============================] - 8s 346ms/step - loss: 0.1350 - acc: 0.9428 - val_loss: 0.1813 - val_acc: 0.9295
Epoch 58/100
24/24 [==============================] - 8s 350ms/step - loss: 0.1457 - acc: 0.9340 - val_loss: 0.1872 - val_acc: 0.9383
Epoch 59/100
24/24 [==============================] - 8s 350ms/step - loss: 0.1399 - acc: 0.9427 - val_loss: 0.1741 - val_acc: 0.9427
Epoch 60/100
24/24 [==============================] - 8s 344ms/step - loss: 0.1265 - acc: 0.9522 - val_loss: 0.1772 - val_acc: 0.9339
Epoch 61/100
24/24 [==============================] - 8s 345ms/step - loss: